<a href="https://colab.research.google.com/github/ykitaguchi77/PreProcess_Image_colab/blob/master/split_dataset_for_crossvalidation_includingTestset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Split datasets for cross validation**
Trainデータセットを5分割し、そのうち1つをvalセットに、残りの合計をtestセットに分割するスクリプト


In [1]:
import random
import glob
import os
import shutil
import numpy as np
import time

from google.colab import drive
drive.mount('/content/drive')


'''
-----orig_data-----grav
                |--cont
↓
↓

-----dst_data[0]------dst_train[0]----grav
  |                |               |-- cont
  |                |--dst_val[0]------grav
  |                                |--cont
  |
  |--dst_data[1]------dst_train[1]----grav
  |                |               |-- cont
  |                |--dst_val[1]------grav
  |                                |--cont
  ...
  |--dst_data[9]------dst_train[9]----grav
  |                |               |-- cont
  |                 |--dst_val[9]------grav
  |                                 |--cont
  |
  |--dst_test---grav
              |--cont
'''

Mounted at /content/drive


'\n-----orig_data-----grav\n                |--cont\n↓\n↓\n\n-----dst_data[0]------dst_train[0]----grav\n  |                |               |-- cont\n  |                |--dst_val[0]------grav\n  |                                |--cont\n  |\n  |--dst_data[1]------dst_train[1]----grav\n  |                |               |-- cont\n  |                |--dst_val[1]------grav\n  |                                |--cont\n  ...\n  |--dst_data[1]------dst_train[9]----grav\n  |                |               |-- cont\n  |                 |--dst_val[9]------grav\n  |                                 |--cont\n  |\n  |--dst_test---grav\n              |--cont\n'

#**Module群**

In [28]:
def get_path(orig_path, dst_path, split_num):
    classes = os.listdir(orig_path) #クラス名を取得
    #データの分割数を設定
    data_list = [0]*len(classes)
    k=0
    for i in classes:
        data_list[k] = glob.glob(orig_path+'/'+i+'/*')
        k+=1
    split_length = int(len(data_list)/split_num)
    return data_list, classes, split_length

def makefolder(orig_path, dst_path, classes):
    #フォルダを作成
    if not os.path.exists(dst_path):  # ディレクトリがなかったら
        os.mkdir(dst_path)  # 作成したいフォルダ名を作成
        for i in range(split_num):
            os.mkdir(dst_path+'/'+str(i))
            os.mkdir(dst_path+'/'+str(i)+'/train')
            os.mkdir(dst_path+'/'+str(i)+'/val')
            for j in classes:
                os.mkdir(dst_path+'/'+str(i)+'/train/'+j)
                os.mkdir(dst_path+'/'+str(i)+'/val/'+j)
        for j in classes:
            os.mkdir (dst_path+'/test/'+j)
            os.mkdir (dst_path+'/test/'+j)


def split_testset(data_list, test_num):
    dst_test = [0] * test_num

    #testsetの指定
    dst_test = random.sample(data_list, test_num)

    #データセット全体とtestsetの差分を取り、dst_dataに格納
    dst_data = [x for x in data_list if x not in dst_test]

    return dst_data, dst_test


def split_data_list(data_list, split_num):
    split_data, dst_data, dst_train, dst_val, dst_test = [0]*split_num, [0]*split_num, [0]*split_num, [0]*split_num, [0]*split_num

    #データの分割
    split_data = list(np.array_split(data_list, split_num))

    #データセット全体と分割したデータの差分を取り、dst_dataに格納

    dst_data = [0] * split_num
    for i in range(split_num):
        dst_data[i] = [x for x in data_list if x not in split_data[i]]

    #トレーニングセット、バリデーションセット、バリデーションセットのリスト作成
    for i in range(split_num):
        dst_train[i] = dst_data[i]
        dst_val[i] = split_data[i]  #バリデーションセット

    return dst_train, dst_val


def copy_to_folders(split_num, class_name, dst_train, dst_val, dst_test, dst_path):
    k=0
    for i in range(split_num):
        dst_path_train = dst_path +'/'+str(i)+'/train/'+class_name
        dst_path_val = dst_path +'/'+str(i)+'/val/'+class_name
        dst_path_test = dst_path +'/test/'+class_name

        for p in dst_train[k]:  # 選択したファイルを目的フォルダにコピー
            shutil.copy(p, dst_path_train)
            #print(p)
            print(dst_path_train)

        for p in dst_val[k]:  # 選択したファイルを目的フォルダにコピー
            shutil.copy(p, dst_path_val)
            #print(p)    
            print(dst_path_val)
        
        for p in dst_test[k]:  # 選択したファイルを目的フォルダにコピー
            shutil.copy(p, dst_path_test)
            #print(p)    
            print(dst_path_test)
        k+=1

def main():
    orig_path = "/content/drive/MyDrive/Deep_learning/666mai_dataset/gravcont_250px"
    dst_path = "/content/drive/MyDrive/Deep_learning/666mai_dataset/crossvalidation_250px"  # フォルダ名
    split_num = 5  #データをいくつに分割するかを記載
    test_num = 43  #1クラスごとのtestsetの枚数を記載

    data_list, classes, split_length = get_path(orig_path, dst_path, split_num)
    makefolder(orig_path, dst_path, classes)

    k=0
    for i in range(len(classes)):
        dst_data, dst_test = split_testset(data_list[k], test_num)
        dst_train, dst_val = split_data_list(dst_data, split_num)
        copy_to_folders(split_num, classes[k], dst_train, dst_val, dst_test, dst_path)
        k+=1

In [29]:
if __name__ == "__main__":
    start = time.time()
    main()
    elapsed_time = time.time() - start
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

FileNotFoundError: ignored

In [21]:
orig_path = "/content/drive/MyDrive/Deep_learning/666mai_dataset/gravcont_250px"
dst_path = "/content/drive/MyDrive/Deep_learning/666mai_dataset/crossvalidation_250px"  # フォルダ名
split_num = 5  #データをいくつに分割するかを記載
test_num = 43  #testsetの枚数を記載

data_list, classes, split_length = get_path(orig_path, dst_path, split_num)
makefolder(orig_path, dst_path, classes)

print(len(dst_data))
k=0
for i in range(len(classes)):
    dst_data, dst_test = split_testset(data_list[k], test_num)
    dst_train, dst_val = split_data_list(dst_data[k], split_num)
    copy_to_folders(split_num, classes[k], dst_train, dst_val, dst_test, dst_path)
    k+=1

247
247
86
2


AxisError: ignored

#**作ったフォルダを削除したいとき**

In [ ]:
dst_path = "/content/drive/My Drive/gravcont_crossvalidation"
directory = dst_path
try:
    shutil.rmtree(directory)
except FileNotFoundError:
    pass

KeyboardInterrupt: ignored